In [1]:
import pandas as pd
import copy
import time
import random
from load_dataset import load
from classifier import NeuralNetwork, LogisticRegression, SVM
from utils import *
from metrics import *  # include fairness and corresponding derivatives
from expl import explanation_candidate_generation, get_top_k_expl
from operator import itemgetter
from influence import *

random.seed(1)
np.random.seed(1)
torch.manual_seed(1)

In [2]:
# ignore all the warnings
import warnings
warnings.filterwarnings('ignore') 

**Load Dataset**

In [3]:
dataset = 'german'
X_train, X_test, y_train, y_test = load(dataset)

In [ ]:
duplicates = 1
make_duplicates = lambda x, d: pd.concat([x]*d, axis=0).reset_index(drop=True)
X_train = make_duplicates(X_train, duplicates)
X_test = make_duplicates(X_test, duplicates)
y_train = make_duplicates(y_train, duplicates)
y_test = make_duplicates(y_test, duplicates)

In [4]:
X_train_orig = copy.deepcopy(X_train)
X_test_orig = copy.deepcopy(X_test)

# Scale data: regularization penalty default: ‘l2’, ‘lbfgs’ solvers support only l2 penalties. 
# Regularization makes the predictor dependent on the scale of the features.
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

**Loss function** (Log loss for logistic regression)

In [5]:
# clf = NeuralNetwork(input_size=X_train.shape[-1])
clf = LogisticRegression(input_size=X_train.shape[-1])
# clf = SVM(input_size=X_train.shape[-1])
num_params = len(convert_grad_to_ndarray(list(clf.parameters())))
if isinstance(clf, LogisticRegression):
    loss_func = logistic_loss_torch
elif isinstance(clf, SVM):
    loss_func = svm_loss_torch
elif isinstance(clf, NeuralNetwork):
    loss_func = nn_loss_torch

**Metrics: Initial state**

In [6]:
clf = LogisticRegression(input_size=X_train.shape[-1])
# clf = NeuralNetwork(input_size=X_train.shape[-1])
# clf = SVM(input_size=X_train.shape[-1])

clf.fit(X_train, y_train)

y_pred_test = clf.predict_proba(X_test)
y_pred_train = clf.predict_proba(X_train)

spd_0 = computeFairness(y_pred_test, X_test_orig, y_test, 0, dataset)
print("Initial statistical parity: ", spd_0)

tpr_parity_0 = computeFairness(y_pred_test, X_test_orig, y_test, 1, dataset)
print("Initial TPR parity: ", tpr_parity_0)

predictive_parity_0 = computeFairness(y_pred_test, X_test_orig, y_test, 2, dataset)
print("Initial predictive parity: ", predictive_parity_0)

loss_0 = logistic_loss(y_test, y_pred_test)
print("Initial loss: ", loss_0)

accuracy_0 = computeAccuracy(y_test, y_pred_test)
print("Initial accuracy: ", accuracy_0)

Initial statistical parity:  -0.09527580118738121
Initial TPR parity:  -0.07785149359511678
Initial predictive parity:  -0.10136869102808022
Initial loss:  0.5078892147492744
Initial accuracy:  0.755


**Select delta fairness function depending on selected metric**

In [7]:
metric = 0  # spd -> 0, tpr parity -> 1, predictive parity -> 2
metric_val = [spd_0, tpr_parity_0, predictive_parity_0][metric]

v1 = get_del_F_del_theta(clf, X_test_orig, X_test, y_test, dataset, metric)

**Pre-compute: (1) Hessian $H_{\theta}$ (2) del_L_del_theta for each training data point**

In [8]:
hessian_all_points = get_hessian_all_points(clf, X_train, y_train, loss_func)

100%|██████████| 800/800 [00:10<00:00, 77.08it/s]


In [9]:
del_L_del_theta = get_del_L_del_theta(clf, X_train, y_train, loss_func)

**Hessian vector product: $H_{\theta}^{-1}v$**

In [10]:
hinv_v, hinv = get_hinv_v(hessian_all_points, v1)

**Removal-based explanation generation**

In [11]:
del_f_threshold = 0.1 * metric_val
support = 0.05  # Do not consider extremely small patterns
support_small = 0.3  # For small patterns, 2nd-order estimation is quite accurate

**Second-order influence computation for a group of points in subset U**

In [12]:
del_f_threshold = (0.1) * metric_val
support = 0.05 # Do not consider extremely small patterns
support_small = 0.3 # For small patterns, 2nd-order estimation is quite accurate
print("del_f_threshold:", del_f_threshold)
print("support:", support)
print("support_small:", support_small)

del_f_threshold: -0.009527580118738123
support: 0.05
support_small: 0.3


In [13]:
attributes = []
attributeValues = []
first_order_influences = []
second_order_influences = []
fractionRows = []

v1_orig = v1
for col in X_train_orig.columns:
    if dataset == 'german':
        if "purpose" in col or "housing" in col: #dummy variables purpose=0 doesn't make sense
            vals = [1]
        else:
            vals = X_train_orig[col].unique()
    elif dataset == 'adult':
        vals = X_train_orig[col].unique()
    elif dataset == 'compas':
        vals = X_train_orig[col].unique()
    elif dataset == 'sqf':
        vals = X_train_orig[col].unique()
    elif dataset == 'random':
        vals = X_train_orig[col].unique()
    else:
        raise NotImplementedError
    for val in vals:
        idx = X_train_orig[X_train_orig[col] == val].index  
        if (len(idx)/len(X_train) > support):  
            y = y_train.drop(index=idx, inplace=False)
            if len(y.unique()) > 1:
                idx = X_train_orig[X_train_orig[col] == val].index
                params_f_2 = second_order_group_influence(idx, del_L_del_theta, hessian_all_points, hinv)
                del_f_2 = np.dot(v1.transpose(), params_f_2)

                attributes.append(col)
                attributeValues.append(val)
                second_order_influences.append(del_f_2)
                fractionRows.append(len(idx)/len(X_train)*100)

In [14]:
expl = [attributes, attributeValues, second_order_influences, fractionRows]
expl = (np.array(expl).T).tolist()

explanations = pd.DataFrame(expl, columns=["attributes", "attributeValues", "second_order_influences", "fractionRows"])
explanations['second_order_influences'] = explanations['second_order_influences'].astype(float)
explanations['fractionRows'] = explanations['fractionRows'].astype(float)

In [15]:
candidates = copy.deepcopy(explanations)
candidates.loc[:, 'score'] = candidates.loc[:, 'second_order_influences']*100/candidates.loc[:, 'fractionRows']
# display(candidates)

In [16]:
%%time
candidates_all = []
total_rows = len(X_train_orig)

# Generating 1-candidates
candidates_1 = []
for i in range(len(candidates)):
    candidate = []
    candidate_i = candidates.iloc[i]
    if ((candidate_i["fractionRows"] >= support_small) or
       ((candidate_i["fractionRows"] >= support) & (candidate_i["second_order_influences"] > del_f_threshold))
       ):
        attr_i = candidate_i["attributes"]
        val_i = int(float(candidate_i["attributeValues"]))
        idx = X_train_orig[X_train_orig[attr_i] == val_i].index
        predicates = [attr_i + '=' + str(val_i)]
        candidate = [predicates, candidate_i["fractionRows"],
                     candidate_i["score"], candidate_i["second_order_influences"], idx]
        candidates_1.append(candidate)

print("Generated: ", len(candidates_1), " 1-candidates")
candidates_1.sort()
# display(candidates_1)

for i in range(len(candidates_1)):
    if (float(candidates_1[i][2]) >= support): # if score > top-k, keep in candidates, not otherwise
        candidates_all.insert(len(candidates_all), candidates_1[i])

# Generating 2-candidates
candidates_2 = []
for i in range(len(candidates_1)):
    predicate_i = candidates_1[i][0][0]
    attr_i = predicate_i.split("=")[0]
    val_i = int(float(predicate_i.split("=")[1]))
    sup_i = candidates_1[i][1]
    idx_i = candidates_1[i][-1]
    for j in range(i):
        predicate_j = candidates_1[j][0][0]
        attr_j = predicate_j.split("=")[0]
        val_j = int(float(predicate_j.split("=")[1]))
        sup_j = candidates_1[j][1]
        idx_j = candidates_1[j][-1]
        if (attr_i != attr_j):
            idx = idx_i.intersection(idx_j)
            fractionRows = len(idx)/total_rows * 100
            isCompact = True
            if (fractionRows == min(sup_i, sup_j)): # pattern is not compact if intersection equals one of its parents
                isCompact = False
            if (fractionRows/100 >= support):
                params_f_2 = second_order_group_influence(idx, del_L_del_theta, hessian_all_points, hinv)
                del_f_2 = np.dot(v1.transpose(), params_f_2)
                score = del_f_2 * 100/fractionRows
                if ((fractionRows/100 >= support_small) or
                    ((score > candidates_1[i][2]) & (score > candidates_1[j][2]))):
                        predicates = [attr_i + '=' + str(val_i), attr_j + '=' + str(val_j)]
                        candidate = [sorted(predicates, key=itemgetter(0)), len(idx)*100/total_rows,
                                    score, del_f_2, idx]
                        candidates_2.append(candidate)
#                         print(candidate)
                        if (isCompact):
                            candidates_all.append(candidate)
print("Generated: ", len(candidates_2), " 2-candidates")

# Recursively generating the rest
candidates_L_1 = copy.deepcopy(candidates_2)
set_L_1 = set()
iteration = 2
while((len(candidates_L_1) > 0) & (iteration < 4)):
    print("Generated: ", iteration)    
    candidates_L = []
    for i in range(len(candidates_L_1)):
        candidate_i = set(candidates_L_1[i][0])
        sup_i = candidates_L_1[i][1]
        idx_i = candidates_L_1[i][-1]
        for j in range(i):
            candidate_j = set(candidates_L_1[j][0])
            sup_j = candidates_L_1[j][1]
            idx_j = candidates_L_1[j][-1]
            merged_candidate = sorted(candidate_i.union(candidate_j), key=itemgetter(0))
            if json.dumps(merged_candidate) in set_L_1:
                continue
            if (len(merged_candidate) == iteration + 1):
                intersect_candidates = candidate_i.intersection(candidate_j)
                setminus_i = list(candidate_i - intersect_candidates)[0].split("=")
                setminus_j = list(candidate_j - intersect_candidates)[0].split("=")
                attr_i = setminus_i[0]
                val_i = int(setminus_i[1])
                attr_j = setminus_j[0]
                val_j = int(setminus_j[1])
                if (attr_i != attr_j):
                    # merge to get L list
                    idx = idx_i.intersection(idx_j)
                    fractionRows = len(idx)/len(X_train) * 100
                    isCompact = True
                    if (fractionRows == min(sup_i, sup_j)): # pattern is not compact if intersection equals one of its parents
                        isCompact = False
                    if (fractionRows/100 >= support):
                        X = np.delete(X_train, idx, 0)
                        y = y_train.drop(index=idx, inplace=False)

                        params_f_2 = second_order_group_influence(idx, del_L_del_theta, hessian_all_points, hinv)
                        del_f_2 = np.dot(v1.transpose(), params_f_2)

                        score = del_f_2 * 100/fractionRows
                        if (((score > candidates_L_1[i][2]) & (score > candidates_L_1[j][2])) or 
                           (fractionRows >= support_small)):
                            candidate = [merged_candidate, fractionRows,
                                         del_f_2*len(X_train)/len(idx), del_f_2, idx]
                            candidates_L.append(candidate)
                            set_L_1.add(json.dumps(merged_candidate))
                            if (isCompact):
                                candidates_all.insert(len(candidates_all), candidate)
    set_L_1 = set()
    print("Generated:", len(candidates_L), " ", str(iteration+1), "-candidates")
    candidates_L_1 = copy.deepcopy(candidates_L)
    candidates_L_1.sort()
    iteration += 1

Generated:  62  1-candidates
Generated:  341  2-candidates
Generated:  2
Generated: 2418   3 -candidates
Generated:  3
Generated: 10779   4 -candidates
CPU times: user 19.4 s, sys: 147 ms, total: 19.5 s
Wall time: 19.8 s


In [17]:
candidates_support_3_compact = copy.deepcopy(candidates_all)
print(len(candidates_support_3_compact))
candidates_df_3_compact = pd.DataFrame(candidates_support_3_compact, columns=["predicates","support","score","2nd-inf",'idx'])
candidates_df_3_compact = candidates_df_3_compact.sort_values(by=['score'], ascending=False)
print(len(candidates_df_3_compact))

13381
13381


**Containment-based filtering**

In [18]:
support_threshold = support_small
containment_df = candidates_df_3_compact[candidates_df_3_compact['support'] < support_threshold*100].sort_values(by=['score'],
                                                                                              ascending=False).copy()

topk = Topk(method='containment', threshold=0.2, k=5)
for row_idx in range(len(containment_df)):
    row = containment_df.iloc[row_idx]
    explanation, score = row[0], row[2]
    topk.update(explanation, score, X_train_orig)
    if len(topk.top_explanations) == topk.k:
        break

**Integrate Results**

In [19]:
explanations = list(topk.top_explanations.keys())
idxs = [v[1] for v in topk.top_explanations.values()]
supports = list()
scores = list()
gt_scores = list()
infs = list()
gts = list()
new_accs = list()
for e in explanations:
    idx = get_subset(json.loads(e), X_train_orig)
    X = np.delete(X_train, idx, 0)
    y = y_train.drop(index=idx, inplace=False)
    clf.fit(np.array(X), np.array(y))
    y_pred = clf.predict_proba(np.array(X_test))
    new_acc = computeAccuracy(y_test, y_pred)
    inf_gt = computeFairness(y_pred, X_test_orig, y_test, 0, dataset) - spd_0
    
    condition = candidates_df_3_compact.predicates.apply(lambda x: x==json.loads(e))
    supports.append(float(candidates_df_3_compact[condition]['support']))
    scores.append(float(candidates_df_3_compact[condition]['score']))
    infs.append(float(candidates_df_3_compact[condition]['2nd-inf']))
    gts.append(inf_gt/(-spd_0))
    gt_scores.append(inf_gt*100/float(candidates_df_3_compact[condition]['support']))
    new_accs.append(new_acc)


expl = [explanations, supports, scores, gt_scores, infs, gts, new_accs]
expl = (np.array(expl).T).tolist()

explanations = pd.DataFrame(expl, columns=["explanations", "support", "score", "gt-score", "2nd-inf(%)", "gt-inf(%)", "new-acc"])
explanations['score'] = explanations['score'].astype(float)
explanations['gt-score'] = explanations['gt-score'].astype(float)
explanations['support'] = explanations['support'].astype(float)
explanations['2nd-inf(%)'] = explanations['2nd-inf(%)'].astype(float)/(-spd_0)
explanations['gt-inf(%)'] = explanations['gt-inf(%)'].astype(float)
explanations['new-acc'] = explanations['new-acc'].astype(float)

pd.set_option('max_colwidth', 100)
explanations.sort_values(by=['score'], ascending=False)


,explanations,support,score,gt-score,2nd-inf(%),gt-inf(%),new-acc
0,"[""age=1"", ""gender=0""]",5.000,0.983225,1.079512,0.515989,0.566519,0.745
1,"[""age=1"", ""credit_hist=0"", ""gender=1""]",6.250,0.620770,0.660626,0.407219,0.433364,0.755
2,"[""credit_amt=0"", ""install_rate=4"", ""install_plans=0"", ""status=0""]",6.500,0.418993,0.413260,0.285849,0.281938,0.755
3,"[""duration=1"", ""employment=4"", ""num_liable=1"", ""residence=4""]",5.375,0.404900,0.403639,0.228425,0.227714,0.740
4,"[""credit_hist=2"", ""gender=0"", ""housing_A152=1"", ""savings=0""]",8.000,0.393087,0.385020,0.330062,0.323289,0.760
